In [1]:
import pandas as pd
from tqdm import tqdm
from nixtla import NixtlaClient

In [2]:
nixtla_client = NixtlaClient(
    api_key = 'nixtla-tok-7BjxcSNBxzLpZmgxkjWkCKJvEZq9Cm5gIeAcRJ3BQHwNSAlDJRK0p3K4W5SfK6jAG405E3xzZ6XIJsvf' # ys api
)
nixtla_client.validate_api_key()

INFO:nixtla.nixtla_client:Happy Forecasting! :), If you have questions or need support, please email ops@nixtla.io


True

In [3]:
start_point = 4000*12
batch_size = 12
num = 1000
output_file = './data/forecasts_4001_5000.csv'

flatten_df = pd.read_csv('./data/flattened_loop_sensor_test_12h.csv')
flatten_df = flatten_df.iloc[start_point :start_point + batch_size * num]
flatten_df['t_1h'] = pd.to_datetime(flatten_df['t_1h'])
print(flatten_df.shape)
flatten_df.head()

(12000, 5)


,id,iu_ac,t_1h,etat_barre,q
48000,4001,24,2023-10-13 16:00:00,2,595.0
48001,4001,24,2023-10-13 17:00:00,2,713.0
48002,4001,24,2023-10-13 18:00:00,2,678.0
48003,4001,24,2023-10-13 19:00:00,2,375.0
48004,4001,24,2023-10-13 20:00:00,2,341.0


In [4]:
num_batches = len(flatten_df) // batch_size
all_forecasts = []

In [5]:
# Forecasting loop
for i in tqdm(range(num_batches), desc='Forecasting'):
    # 读取12条数据
    df = flatten_df.iloc[i * batch_size:(i + 1) * batch_size]
    
    # 进行预测
    timegpt_fcst_df = nixtla_client.forecast(
        df=df, 
        h=1,  # 预测步长
        # finetune_steps=10,
        # finetune_loss='mae',   
        freq='h', 
        time_col='t_1h', 
        target_col='q', 
        model='timegpt-1'
    )

    # Add index starting from 1 + start_point
    timegpt_fcst_df.reset_index(drop=True, inplace=True)
    timegpt_fcst_df.index += 1 + int(start_point/12) + i

    # Save the current forecast to the output CSV file
    if i == 0:
        # For the first batch, write the header
        timegpt_fcst_df.to_csv(output_file, index=True, index_label='id', mode='w')
    else:
        # For subsequent batches, append without the header
        timegpt_fcst_df.to_csv(output_file, index=True, index_label='id', mode='a', header=False)

    # Append the current forecast to the list
    all_forecasts.append(timegpt_fcst_df)

# Optionally, you can still combine all forecasts into one DataFrame and perform additional operations if needed
all_forecasts_df = pd.concat(all_forecasts, ignore_index=False)
print(all_forecasts_df.shape)
print(all_forecasts_df.head())


Forecasting:   0%|          | 0/1000 [00:00<?, ?it/s]INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
Forecasting:   0%|          | 1/1000 [00:03<1:04:24,  3.87s/it]INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
Forecasting:   0%|          | 2/1000 [00:06<55:52,  3.36s/it]  INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
Forecasting:   0%|          | 3/1000 [00:10<57:06,  3.44s/it]INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INF

(1000, 2)
                      t_1h     TimeGPT
48001  2023-10-14 04:00:00   69.662369
48013  2023-10-15 09:00:00  139.344757
48025  2023-10-16 14:00:00  346.136230
48037  2023-10-17 19:00:00  325.045349
48049           2023-10-19  277.565063
